# Dependencias

In [ ]:
# Para instalar el paquete
%pip install uncertainties

In [ ]:
import numpy as np
import uncertainties as un
from uncertainties.umath import sin as unsine
import csv
import os 

# Funciones

In [121]:
## La función split redondeado agarra un nunmero de la forma ufloat
## y te devuelve su error a una cifra significativa y su valor nominal 
## con la misma cantidad de cifras.
def split_redondeado(x,tipo):
    # Usar str() con formato especial si está disponible
    s = f"{x:.1u}"

    # Separar en valor nominal y error
    valor_str, error_str = s.split("+/-")
    if (tipo == "str"):
        return valor_str,error_str
    elif (tipo == "float"):
        return un.ufloat(float(valor_str),float(error_str))

## La función deg2ra toma un valor tito en la forma = [°,',''] 
## y te lo pasa a titoradianes con un error de +-15''
def deg2ra(tito):
    tito_total = tito[0]+tito[1]/60+ tito[2]/3600
    return un.ufloat(tito_total,15/3600) * np.pi/180

## La función lam toma un valor de ángulo, la amplitud de la rendija y el número m
def lam(tito,tito_0,d,m):
    Resultado = []
    for valor in tito:
        Resultado.append(unsine(tito_0-valor)*d/m)
    return Resultado

## Esta función crea la matriz de los datos
def medición():
    # Esta lista permite distinguir entre grados minutos y segundos
    orden = ["Grados","Minutos","Segundos"]

    # En esta lista se van a almacenar todos los valores de los angulos convertidos a radianes
    Lista = []

    # Esta variable centinela nos permite entrar en el bucle while.
    centinela = 0

    # La variable i es el contador de la medición.
    i=0
    
    while centinela != -1:
        
        i+=1
        j=0
        # La lista "angulo_grados" sirve para guardar los °,','' del ángulo dato.
        angulo_grados = []  
        
        # Un bucle para almacenar los 3 datos en "angulo_grados".
        while (j<=2):
            centinela = input(f"Ingrese el valor de {orden[j]} de la medición {i}: \nSi quieres rehacer la medición {i}? presioná e\nSi quieres salir, buena suerte, presiona s")
            
            j+=1

            # Si queres salir
            if (centinela in "sS"):
                centinela = -1
                angulo_grados = 0
                j = 3
            # Para rehacer el angulo_grados i hay que retroceder el j a 0 y dejar angulo_grados vacío
            elif(centinela in "eE"):
                j=0
                angulo_grados = 0
            else:
                angulo_grados.append(float(centinela))
        
        # Conversión de grados a rad.
        if (angulo_grados!=0):
            Lista.append(deg2ra(angulo_grados))  
    
        # Le comunico al usuario en que medición esta
        if (centinela!=-1):
            print(f"se terminó la medición N°{i} = {angulo_grados[0]}° {angulo_grados[1]}' {angulo_grados[2]}''")
        else:            
            print(f'se terminaron tomando N°{i-1} mediciones')
    # Paso todo a un array
    return np.array(Lista)

## La función guardar_csv guarda los datos en un .csv con el nombre elegido 
def guardar_csv(matriz, nombre_elemento, columnas):

    ruta = nombre_elemento 
    
    # Verificar si la carpeta existe
    
    if not os.path.exists(ruta):
    
        os.makedirs(ruta)  # Crea la carpeta y subcarpetas si es necesario
    
        print(f"Carpeta '{ruta}' creada")
    
    else:
    
        print(f"Carpeta '{ruta}' ya existe")

#Lo que hace la virable nombre_archivo es:
#                    carpeta / nombre del elemento _ Lambda o Radianes .csv 
    nombre_archivo = ruta + "/"+nombre_elemento + "_" + columnas[0] + ".csv"

    with open(nombre_archivo, mode="w", newline="") as file:
        writer = csv.writer(file)
        
        # Escribir encabezado
        writer.writerow(["Nº",columnas[0],columnas[1]])
        
        # Escribir filas numeradas
        for i, fila in enumerate(matriz, start=1):
            valor_nominal,error_estandar = split_redondeado(fila,"str")
            writer.writerow([i] + [valor_nominal, error_estandar])

    print(f"Archivo '{nombre_archivo}' guardado correctamente.")

## Esta es una función para mostrar los resultados en tablas de los radianes o los lambdas
## mostrando sus valores nominales y sus incertidumbres
def mostrar_resultados(labelsx, data):
    # Creo las etiquetas del eje vertical en función de la cantidad de medidas
    labelsy = [f"N°{i}" for i in range(1,len(data)+1)]

    print("{:^{}} | {:^{}} | {:^{}}".format(
    # Imprimimos espacios en blanco de acuerdo a la longitud del último N medido (puede ser N° 10)
        '', len(labelsy[-1]), 

    # Imprimimos la primera etiqueta del eje x
        labelsx[0], len(labelsx[0]), 

    # Imprimimmos la segunda etiqueta del eje x
        labelsx[1], len(labelsx[1])))

    for i in range(len(data)):
        # Extraemos los datos de la medición "N°i" y los dividimos en su valor nominal y su error
        valor_nominal, error_estandar = split_redondeado(data[i], "str")
        # el ^ es una acomodamiento dinamico y permite centrar el valor
        print("{:^{}} | {:^{}} | {:^{}}".format(

    # Imprimimos el "N°i", dejando el espacio del último N°
            labelsy[i]   , len(labelsy[-1]), 

    # Imprimimos el valor nominal con la longiud de la etiqueta    
            valor_nominal, len(labelsx[0]), 

    # Idem que el anterior
            error_estandar, len(labelsx[1])))

# Funcionamiento principal


Para que funcione el codigo debes ejecutar las dependencias, las funciones y la celda de abajo.
### ¿Qué hace?
Muchas cosas.
Antes de empezar ES IMPORTANTE modifificar el valor de tito_0 que sería tu 0 angular.

El programa pide diferente mediciones del ángulo en la forma x° y' z'' y la transforma a radianes con una incertdumbre por defecto de 15''.

Luego, guarda los resultados en un csv en una carpeta con el nombre del elemento.

Luego, hace los cálculos de $\lambda$ teniendo como predeterminado $ m = 1 $ y hace el mismo procedimiento de guardado que para los ángulos.

Para acceder a los datos hay dos opciones:
- Usar la variable Lambda_medido con la función split_redondeado(Lambda_medido[i],"float")
- Acceder desde el csv usando una función que no quise hacer así que si de casualidad la haces estaría muy agradecido que la subas a https://github.com/FabriTape/Exp-2-Lab-2-Emision-de-gases-

In [ ]:
## Este valor de tito 0 esta puesto pero haria que cambiarlo en caso de ser necesario
tito_0 = deg2ra([216,20,15])

## Empezamos con el nombre del elemento al que le medimos las lineas
Nombre = input("ingrese el nombre del elemento")

## Inicio el programa de adquisició de angulos y transformación a radianes
Angulos = medición()

## Guardo los ángulos en un csv
guardar_csv(Angulos,Nombre,["Radianes","Error"])

Lambda_medido = []

## Ancho de las rendijas usando 300 lineas/mm y pasando a nm
d = 10000/3 

## Usamos el máximo principal
m = 1

## Hacer el calculo para el lambda
Lambda_medido = lam(Angulos,tito_0,d,m)

guardar_csv(Lambda_medido,Nombre,["Lambda","Error"])


## Mostrar resultados

print(f"{"="*39}\n Mediciones de los Ángulos en radianes\n{"="*39}")

mostrar_resultados(["Radianes","Error_estandar"],Angulos)

print(f"\n{"="*32}\n Resultados de los Lambda en nm\n{"="*32}")

mostrar_resultados(["Longitud de onda","Error_estandar"],Lambda_medido)

